<a href="https://colab.research.google.com/github/Aaradhy8989/colab/blob/main/Template_Cricket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
# Students will submit their files with their team-name.py 
# Student have to use the Team as their parent class

In [ ]:
class Team:
  def __init__(self,self_features,opponent_features):
    self.self_features = self_features
    self.opponent_features = opponent_features
    self.explore_wicket = 0
    self.explore_runs = 0

  def get_next_batter(self):
    return np.random.randint(0,5)

  def get_next_bowler(self):
    return np.random.randint(0,5)

  def get_batting_action(self,ball,total_runs,wickets_left,score_to_chase,feature_batter,feature_bowler):
    batting_action = np.random.randint(0,6)
    return batting_action

  def get_bowling_action(self,ball,total_runs,wickets_left,score_to_chase,feature_batter,feature_bowler):
    bowling_action = np.random.randint(0,3)
    return bowling_action

  def get_explore_actions(self):
    batter = np.random.randint(0,5)
    bowler = np.random.randint(0,5)
    batting_action = np.random.randint(0,6)
    bowling_action = np.random.randint(0,3)
    feature_batter = self.self_features[batter,0:2]
    feature_bowler = self.self_features[bowler,2:4]
    return feature_batter,feature_bowler, batting_action, bowling_action  

  def set_explore_outcomes(self,wicket,runs):
    self.explore_wicket = wicket
    self.explore_runs = runs
  
  def explore_compute(self):
    pass

  def explore_dp(self):
    pass


class Australia(Team):
  pass

class India(Team):
  pass


In [ ]:
class Match:

  def __init__(self,num_balls, explore_num_balls,action_timeout, explore_timeout, TeamOne, TeamTwo):
    self.num_balls = num_balls
    self.explore_num_balls = explore_num_balls
    self.action_timeout = action_timeout
    self.explore_timeout = explore_timeout
    self.team_one_features = np.random.uniform(1,5,size=(5,4))
    self.team_two_features = np.random.uniform(1,5,size=(5,4))
    self.team_one = TeamOne(self.team_one_features,self.team_two_features)
    self.team_two = TeamTwo(self.team_two_features,self.team_one_features)
    self.current_batters_list =np.array([1,1,1,1,1])
    self.current_bowlers_list =np.array([2,2,2,2,2])
    self.num_miss_team_batting = 0
    self.num_miss_team_bowling = 0


  def explore_phase_team(self,team_id):
    if (team_id == 1):
      team = self.team_one
    else:
      team = self.team_two
    start_time      = time.time()
    for balls in range(self.explore_num_balls):
      feature_batter,feature_bowler, batting_action, bowling_action = team.get_explore_actions()
      wicket, runs   = self.get_outcome_linear(feature_batter, feature_bowler, batting_action, bowling_action)
      team.set_explore_outcomes (wicket,runs)
      team.explore_compute()
    team.explore_dp()    
    end_time = time.time()    
    if(end_time - start_time > self.explore_timeout):  
      print("Timing Violation During Exploration Phase")
      

  def explore_phase(self):
    self.explore_phase_team(1)
    self.explore_phase_team(2)


  def get_valid_bowler(self,next_bowler):
    if (self.current_bowlers_list[next_bowler]==0):
      print("Bowler Invalid, Choosing Random Bowler")
      bowlers_with_overs_left = np.where(self.current_bowlers_list>0)[0]
      next_bowler = np.random.choice(bowlers_with_overs_left)
    return next_bowler


  def get_valid_batter(self,next_batter):
    if (self.current_batters_list[next_batter]==0):
      print("Batter Invalid, Choosing Random Batter")
      batters_not_out = np.where(self.current_batters_list>0)[0]
      next_batter = np.random.choice(batters_not_out)
    return next_batter

  def next_batter(self,team_batting):
    next_batter = team_batting.get_next_batter()
    next_batter = self.get_valid_batter(next_batter)
    feature_batter = team_batting.self_features[next_batter,0:2]
    return next_batter,feature_batter


  def next_bowler(self,team_bowling):
    next_bowler = team_bowling.get_next_bowler()
    next_bowler = self.get_valid_bowler(next_bowler)
    feature_bowler = team_bowling.self_features[next_bowler,2:4]
    return next_bowler, feature_bowler

  def get_team_batting_action(self,team_batting,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler):
    start_time      = time. time()
    batting_action  = team_batting.get_batting_action(ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
    end_time        = time. time()
    if(end_time - start_time > self.action_timeout):
      batting_action = 0 #this is the default option, we have to fix the penalisation strategy
      self.num_miss_team_batting = self.num_miss_team_batting + 1
    return batting_action


  def get_team_bowling_action(self,team_bowling,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler):
    start_time      = time. time()
    bowling_action  = team_bowling.get_bowling_action(ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
    end_time        = time. time()
    if(end_time - start_time > self.action_timeout):
      bowling_action = 0 #this is the default option, we have to fix the penalisation strategy
      self.num_miss_team_bowling = self.num_miss_team_bowling + 1
    return bowling_action

  def get_outcome_random(self,feature_batter, feature_bowler, batting_action, bowling_action):
    wicket = np.random.randint(0,1)
    runs = np.random.randint(0,7)
    return wicket, runs


  def innings(self,innigins_id,runs_to_chase):
    total_runs = 0
    wickets_left = 5
    self.current_batters_list =np.array([1,1,1,1,1])
    self.current_bowlers_list =np.array([2,2,2,2,2])
    self.num_miss_team_batting = 0
    self.num_miss_team_bowling = 0
    if (innigins_id == 1 ):
      team_batting = self.team_one
      team_bowling = self.team_two
    else:
      team_batting = self.team_two
      team_bowling = self.team_one
    # Initialising the first batter and first bowler
    batter,feature_batter = self.next_batter(team_batting)
    bowler, feature_bowler = self.next_bowler(team_bowling)
    
    for ball in range(self.num_balls):
      if np.sum(self.current_batters_list) > 0 :
        batting_action = self.get_team_batting_action(team_batting,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
        bowling_action = self.get_team_bowling_action(team_bowling,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler)
        wicket, runs   = self.get_outcome_linear(feature_batter, feature_bowler, batting_action, bowling_action)
        total_runs     = total_runs + runs
        if (wicket > 0):
          self.current_batters_list[batter] = 0
          if(np.sum(self.current_batters_list) > 0 ):
            batter,feature_batter = self.next_batter(team_batting)
        if ((ball+1)%6 ==0 ):
          self.current_bowlers_list[bowler] = self.current_bowlers_list[bowler]-1
          if(np.sum(self.current_bowlers_list) > 0 ) :
            bowler, feature_bowler = self.next_bowler(team_bowling)

    return total_runs, self.current_batters_list, self.current_bowlers_list, self.num_miss_team_batting, self.num_miss_team_bowling


In [ ]:
# setting time outs
explore_timeout = 10
action_timeout = 10

explore_num_balls = 60
num_balls = 60
match = Match(num_balls, explore_num_balls,action_timeout, explore_timeout,Australia,India)
match.explore_phase()
first_innings_score, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(1,float('inf'))
second_innings_score, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(2,first_innings_score)